In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tqdm import tqdm
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings("ignore")

In [ ]:
DIR = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset' 
classes = [i for i in os.listdir(DIR) if '.' not in i] # readme dosyası gelmemesi için if eklendi.                  
classes

In [ ]:
# Klasör yolu ve etiketleri tutacak listeleri tanımla
labels = [] # balık türleri saklanacak
paths = [] # görüntü dosyalarının yolları saklancak

# Klasörleri ve dosyaları gez
for root_dir, _, files in os.walk(DIR):
    for file in files:
        # Sadece '.png' uzantılı dosyalarla ilgilen
        if file.endswith('.png'):
            # 'GT' klasöründe olmayan dosyalar için etiket ve yol ekle
            if 'GT' not in root_dir.split():
                # Klasör adını etiket olarak al ve dosya yolunu ekle
                labels.append(os.path.basename(root_dir))
                paths.append(os.path.join(root_dir, file))

# Etiket ve yolları bir DataFrame'e aktarma
df = pd.DataFrame({
    'path': paths,
    'label': labels
})

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['label'].value_counts()

In [ ]:
df.head(10)#Bu kod dataframe ilk 10 satırını gösteriyor

In [ ]:
# Seaborn ile etiketlerin dağılımını gösteren bir histogram
plt.figure(figsize=(10,6))
sns.countplot(x='label', data=df, palette='viridis')

# Grafiğe başlık ve etiketler ekle
plt.title('Balık Türlerine Göre Dağılım', fontsize=16)
plt.xlabel('Balık Türleri', fontsize=14)
plt.ylabel('Adet', fontsize=14)

# X ekseni etiketlerinin açısını değiştir (daha kolay okunabilir hale getirmek için)
plt.xticks(rotation=45, ha='right', fontsize=12)

# Grafiği göster
plt.tight_layout()
plt.show()




### Her bir balık türü için örnek görüntüye bakalım..

In [ ]:
plt.figure(figsize=(15, 12))
for idx, label in enumerate(df['label'].unique()):
    plt.subplot(3, 3, idx + 1)
    plt.imshow(plt.imread(df[df['label'] == label].iloc[0, 0]))
    plt.title(label)
    plt.axis('off')
plt.show()

###  Derin öğrenme modeli eğitmek için gerekli olan yapılandırılmış bir veri seti oluşturma

In [ ]:
main_directory = "/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset"
images = []
labels = []
for directory in tqdm(os.listdir(main_directory)):
    next_directory = f"{main_directory}/{directory}" #Bu if koşulu, veri setinde bulunan README, lisans ve örnek segmentasyon dosyalarını atlamak için kullanılıyor
    if directory in ["README.txt", "license.txt", "Segmentation_example_script.m"]:
        continue
    i = 0
    for images_directory in os.listdir(next_directory):
        # "GT" klasörünün yalnızca referans etiketleri içerir, bu etiketler eğitim verisi olarak kullanılmaz,
        # modelin doğruluğunu kontrol etmek için kullanılır.
        if "GT" not in images_directory:
            final_directory = f"{next_directory}/{images_directory}"
            for image in os.listdir(final_directory):
                # images.append(keras.utils.img_to_array(keras.utils.load_img(f"{final_directory}/{image}")))
                # labels.append(images_directory)
                images.append(cv2.resize(cv2.imread(f"{final_directory}/{image}"),(224, 224)).astype('float32') / 255) 
                #derin öğrenme modellerinin sabit boyutlarda görüntülerle çalışmasını sağlamak için yapılır.
                #Görüntü verisi 0 ile 1 arasında normalize edilir.
                #RGB görüntüleri genellikle 0 ile 255 arasında bir değer aralığına sahiptir, bu yüzden 255'e bölünerek her piksel değeri 0-1 aralığına indirgenir.
                labels.append(images_directory)
        i+=1

In [ ]:
# Görüntü verisinin doğruluğunu kontrol etmek:
print(images[0]) #İmages listesinin ilk elemanını yazdırır.

In [ ]:
print(images[0].shape)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, random_state=0, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, random_state=0, test_size = 0.5)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)
datagen.fit(X_train)

In [ ]:
print(type(X_train))
print(X_train[0].shape)
print(X_val[0].shape)
print(X_test[0].shape)
len(X_train)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
print(y_train)

In [ ]:
# OneHotEncoder nesnesini oluştur
encoder = OneHotEncoder(sparse=False)

# Eğitim,doğrulama ve test etiketlerini one-hot encoding ile dönüştür
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_val = encoder.transform(y_val.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

In [ ]:
print(y_train)

### Hiperparametre Optimizasyonu:

In [ ]:
# Optimizer ve dropout oranını değiştir
model = tf.keras.models.Sequential([
  # Giriş katmanı 
  tf.keras.layers.Flatten(input_shape=(224, 224, 3)),
    
  # 1. katman
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.1),
    
  # 2. katman
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.1),
    
  # 3. katman  
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.1),

  # Çıkış katmanı
  tf.keras.layers.Dense(9,activation="softmax")
])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) #Optimizer derlenmesi 
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])#Modelin derlenmesi 

model.summary()

### Model Eğitimi

In [ ]:
results = model.fit(X_train, y_train, #Modelin Eğitimi
                    batch_size=128,
                    epochs=20,
                    callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=3,
                            restore_best_weights=True
                        )
                    ],
                    validation_data=(X_val, y_val))

### Accuracy - Loss Grafiği

In [ ]:
import matplotlib.pyplot as plt

def plot_training_results(history):
    """Eğitim sonuçlarını görselleştiren fonksiyon."""
    plt.figure(figsize=(12, 5))

    # Loss grafiği
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss', color='blue')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
    plt.title('Loss Grafiği')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Accuracy grafiği
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy', color='green')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='yellow')
    plt.title('Accuracy Grafiği')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Modelin eğitimini gerçekleştirdikten sonra fonksiyonu çağır
plot_training_results(results)

### Test sonuçlarının değerlendirilmesi

In [ ]:
# Test seti ile tahmin yapalım
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

In [ ]:
# Test seti üzerinde modeli değerlendirelim
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

In [ ]:
test_labels = encoder.inverse_transform(y_test)
prediction_labels = encoder.inverse_transform(y_pred)
print(classification_report(test_labels, prediction_labels))